# Conversation between DialogueGpt and Leolani

In this notebook, we create a loop in which DialogGpt has a conversation with Leolani. We send Leolani's initial prompt to DialogueGpt to start a conversation.
Next we capture Leolani's response from the brain and DialogueGpt's response constinuously until we meet the stop condition. Extracted triples are posted to a brain called DialueGpt.

In principle, this conversation can go on forever. At the end, we save the scenario in EMISSOR.

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint
import spacy

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#!python -m spacy download en

### Import the chatbot utility functions

In [3]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Import a conversation agent pipeline

In [4]:
#from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, AutoModelWithLMHead
#import torch

#tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
#model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')

#tokenizer = AutoTokenizer.from_pretrained('gpt2')
#model = AutoModelForCausalLM.from_pretrained('gpt2')

#tokenizer = AutoTokenizer.from_pretrained("manueltonneau/bert-base-cased-conversational-nli")
#model = AutoModel.from_pretrained("manueltonneau/bert-base-cased-conversational-nli")

#tokenizer = AutoTokenizer.from_pretrained("xlnet-large-cased")
#model = AutoModelForCausalLM.from_pretrained("xlnet-large-cased")

#tokenizer = AutoTokenizer.from_pretrained("t5-small")

#model = AutoModelWithLMHead.from_pretrained("t5-small")

#### Needed to suppress messages from DialgGPT
#import os
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)
UTTERANCE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([UTTERANCE], return_tensors='pt')
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))


["<s> That's unfortunate. Are they trying to lose weight or are they just trying to be healthier?</s>"]


## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [6]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "BLENDERBOT"
HUMAN_ID = "BLENDERBOT"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-12-29-20:24:30 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [7]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/blender",
                                log_dir=log_path,
                                clear_all=False)

2021-12-29 20:24:30,084 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-12-29 20:24:30,092 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-12-29 20:24:30,094 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-12-29 20:24:30,096 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-12-29 20:24:30,098 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-12-29 20:24:30,454 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [8]:
replier = LenkaReplier()

2021-12-29 20:24:30,458 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [9]:
chat = Chat(HUMAN_ID)

2021-12-29 20:24:30,463 -     INFO - cltl.triple_extraction.api.Chat (BLENDERBOT)        000 - << Start of Chat with BLENDERBOT >>


In [10]:
nlp = spacy.load("en_core_web_sm")
#nlp= spacy.load('en') # other languages: de, es, pt, fr, it, nl

## Start the interaction

In [11]:
print_details=True

max_context=500
t1 = datetime.now()

#### Initial prompt by the system from which we create a TextSignal and store it
leolani_prompt = f"{choice(TALK_TO_ME)}"
print('\n\t'+AGENT + ": " + leolani_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)


#BLENDERBOT
bot_input_ids = tokenizer(leolani_prompt, return_tensors='pt')
chat_history_ids = model.generate(**bot_input_ids)

# encode the new Leolani input, add the eos_token and return a tensor in Pytorch
##bot_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
# append the new user input tokens to the chat history
#bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if step > 0 else new_input_ids
# generated a response while limiting the total chat history to max_context tokens, 
##chat_history_ids = model.generate(bot_input_ids, max_length=max_context, pad_token_id=tokenizer.eos_token_id)
repetition = []
utterance = ""
response_json = None
#### Get input and loop
while (datetime.now()-t1).seconds <= 3600:
    ###### Getting the next input signals
    #utterance = format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))
    # BLENDER
    utteranceList = tokenizer.batch_decode(chat_history_ids)
    #["<s> Sure, what do you do for a living? I'm an accountant, what about you?</s>"]
    answer = utteranceList[0].strip('</s>')
    #print("ANSWER", answer)
    doc = nlp(answer)
    sentences = []
    for s in doc.sents:
        sentence = ""
        for token in s:
            sentence += token.text+" "
        sentences.append(sentence) 
#    utterance = sentences[len(sentences)-1]
   #print(sentences)                       
    #print("UTTERANCE", utterance)
 
    for utterance in sentences:
        if utterance in repetition:
            print('Repeating', utterance)
            utterance = None
        else:
            repetition.append(utterance)
            
        if not utterance:
            if response_json:
                leolani_prompt =  replier.reply_to_statement(response_json, proactive=True, persist=True)
            else:
                leolani_prompt = f"{choice(TALK_TO_ME)}"

            print('\n\t'+AGENT + ": " + leolani_prompt)
            textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
            scenario_ctrl.append_signal(textSignal)
            
            #BLENDERBOT
            bot_input_ids = tokenizer(leolani_prompt, return_tensors='pt')
            chat_history_ids = model.generate(**bot_input_ids)


            # encode the new Leolani input, add the eos_token and return a tensor in Pytorch
            ##new_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
            # append the new user input tokens to the chat history
            ##bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
            # generated a response while limiting the total chat history to max_context tokens, 
            ##chat_history_ids = model.generate(bot_input_ids, max_length=max_context, pad_token_id=tokenizer.eos_token_id)
        else:
            print('\n\t'+HUMAN_NAME + ": " + utterance)
            textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, HUMAN_ID)
            scenario_ctrl.append_signal(textSignal)

            #### Process input and generate reply

            capsule, leolani_prompt, response_json = talk.process_text_and_reply(scenario_ctrl,
                                   place_id,
                                   location,
                                   HUMAN_ID,
                                   textSignal,
                                   chat,
                                   replier,
                                   my_brain,
                                   print_details)

            if not capsule:                
                replies, response_json = talk.process_triple_spacy_and_reply(scenario_ctrl,
                                   place_id,
                                   location,
                                   HUMAN_ID,
                                   AGENT,
                                   textSignal,
                                   chat,
                                   replier,
                                   my_brain,
                                   nlp,
                                   print_details)
                if print_details:
                    print('Replies:', replies)
                for reply in replies:
                    leolani_prompt+= reply+" . "


            if not capsule:
                replies, response_json = talk.process_text_spacy_and_reply(scenario_ctrl,
                                   place_id,
                                   location,
                                   HUMAN_ID,
                                   AGENT,
                                   textSignal,
                                   chat,
                                   replier,
                                   my_brain,
                                   nlp,
                                   print_details)
                if print_details:
                    print('Replies:', replies)
                for reply in replies:
                    leolani_prompt+= reply+" . "

            print('\n\t'+AGENT + ": " + leolani_prompt)
            textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
            scenario_ctrl.append_signal(textSignal)
            
            #BLENDERBOT
            bot_input_ids = tokenizer(leolani_prompt, return_tensors='pt')
            chat_history_ids = model.generate(**bot_input_ids)


        # encode the new Leolani input, add the eos_token and return a tensor in Pytorch
        ##new_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
        # append the new user input tokens to the chat history
        ##bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        # generated a response while limiting the total chat history to max_context tokens, 
        ##hat_history_ids = model.generate(bot_input_ids, max_length=max_context, pad_token_id=tokenizer.eos_token_id)



	Leolani2: Do you have any gossip?

	BLENDERBOT:   I do , but I try not to let it get to me . 
2021-12-29 20:24:35,836 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-12-29 20:24:35,837 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-12-29 20:24:35,868 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-12-29 20:24:36,724 -     INFO - cltl.triple_extraction.api.Chat (BLENDERBOT)        001 - BLENDERBOT: "I do , but I try not to let it get to me ."


Couldn't parse input


Last utterance:   I do , but I try not to let it get to me . 
Triples subj - aux - amod []
Triples []
Replies: []
Entities []
Subject_and_objects []
Replies: []

	Leolani2: Wise words by wise men write wise deeds in wise pen

	BLENDERBOT: I know it 's not right 
2021-12-29 20:24:41,609 -     INFO - cltl.triple_extraction.api.Chat (BLENDERBOT)        002 - BLENDERBOT: "I know it 's not right"


Couldn't parse input


Last utterance: I know it 's not right 
Triples subj - aux - amod []
Triples []
Replies: []
Entities []
Subject_and_objects []
Replies: []

	Leolani2: Many of us feel that way

	BLENDERBOT: but I ca n't help it . 
2021-12-29 20:24:46,828 -     INFO - cltl.triple_extraction.api.Chat (BLENDERBOT)        003 - BLENDERBOT: "but I ca n't help it ."


...

Couldn't parse input


Last utterance:   I know , right ? 
Triples subj - aux - amod []
Triples []
Replies: []
Entities []
Subject_and_objects []
Replies: []

	Leolani2: So ... it   has come to this

	BLENDERBOT:   It 's crazy to think that it 's so old ! 
2021-12-29 20:38:09,923 -     INFO - cltl.triple_extraction.api.Chat (BLENDERBOT)        078 - BLENDERBOT: "It 's crazy to think that it 's so old !"


Couldn't parse input


Last utterance:   It 's crazy to think that it 's so old ! 
Triples subj - aux - amod []
Triples []
Replies: []
Entities []
Subject_and_objects []
Replies: []

	Leolani2: Now I have seen this too
Repeating   I was so shocked . 

	Leolani2: Tell me anything, I love learning things
Repeating   I thought I was going to have a heart attack . 

	Leolani2: Do you have any gossip?
Repeating   I do , but I try not to let it get to me . 

	Leolani2: Do you have any gossip?
Repeating I know it 's not right 

	Leolani2: Would you like to chat? I'll do my best to keep up
Repeating but I ca n't help it . 

	Leolani2: Tell me something, anything!
Repeating   Well , I just got a promotion at work , so I 'm pretty stoked about that ! 

	Leolani2: Tell me anything, I love learning things
Repeating   Well , I 'm a big fan of learning new things as well . 

	Leolani2: Do you have any gossip?
Repeating   What is your favorite thing to learn ? 

	Leolani2: Would you like to chat? I'll do my best to keep up

KeyboardInterrupt: 

## Save the scenario data

In [12]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook